In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D, AvgPool2D
import matplotlib.pyplot as plt

# Any results you write to the current directory are saved as output.

In [ ]:
x=[]
y=[]
pos=0
neg=0
df = pd.read_csv("/kaggle/input/train.csv",index_col='id')
for filename in os.listdir('/kaggle/input/train/train/'):
    im = np.array(cv2.imread('/kaggle/input/train/train/'+filename))
    lab=int(df.loc[filename])
    
    if lab==1:
        pos+=1
        x.append(im)
        y.append(lab)
    else:
        x.append(im)
        y.append(lab)
        x.append(cv2.flip( im, 0 ))
        y.append(lab)
        x.append(cv2.flip( im, 1 ))
        y.append(lab)
        neg+=3      


print(pos,neg)


In [ ]:
plt.bar(['positive','negative'], [pos,neg])

In [ ]:
x=(np.array(x)/255).reshape(-1,32,32,3)
y=np.array(y).reshape(-1,1)


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.5,decay=1e-5),
              metrics=['accuracy'])
model.summary()


**Training the model**

In [ ]:
model.fit(x, y,
          batch_size=64,
          epochs=20,
          verbose=1,
          validation_split=0.3)

In [ ]:
test_dict ={'id':[],'has_cactus':[]} 
for filename in os.listdir('/kaggle/input/test/test/'):
    #print(filename)
    
    img = (np.array(cv2.imread('/kaggle/input/test/test/'+filename))/255).reshape(1,32,32,3)
    pred = model.predict(img)[0][0]
    test_dict['id'].append(filename)
    test_dict['has_cactus'].append(pred)
    #print(pred[0][0])
df_test = pd.DataFrame.from_dict(test_dict)
df_test.to_csv('test.csv',index=False)